# Datababse Syncronization Script - Palladium 
#### About Script
The script should do the following;
- Get all the required DB names
- Query all the table names in the database
- Merge all the similar tables in the databases
- Create a new database
- dump all the merged tables into the new database. 

#### Import Dependancies


In [1]:
import pandas as pd 
import pymysql
import mysql.connector 
from sqlalchemy import create_engine 

#### Get Databases


In [3]:
host = "localhost"
user = "root" 
password = "P@55w0rd."
auth_plugin = "mysql_native_password"

condb = mysql.connector.connect(
    host=host,
    user=user,
    password=password, 
    auth_plugin=auth_plugin
)

cursor = condb.cursor() 
cursor.execute("show databases")

databases = cursor.fetchall()
 
databases = ' '.join(str(v) for v in databases)
databases = "".join(filter(lambda x: x != "(", databases)) 
databases = "".join(filter(lambda x: x != ")", databases)) 
databases = "".join(filter(lambda x: x != ",", databases))
databases = "".join(filter(lambda x: x != "'", databases)) 

databases = databases.split()
excluded_databases = ['information_schema', 'dwapi', 'mysql', 'testdb']
for database in databases:
    if database not in excluded_databases: 
        print (database)

emrfix
gucha_daily
gucha_migrated
openmrs
performance_schema
sys


#### Define Databases

In [42]:
db_one = "gucha_daily"
db_two = "gucha_migrated"

#### Get Tables in Databases To Be Merged

#####  Tables in Database 1

In [43]:
cursor = condb.cursor()  
cursor.execute("use %s" % db_one)
cursor.execute("show tables")

db1_tables = cursor.fetchall()
 
db1_tables = ' '.join(str(v) for v in db1_tables)
db1_tables = "".join(filter(lambda x: x != "(", db1_tables)) 
db1_tables = "".join(filter(lambda x: x != ")", db1_tables)) 
db1_tables = "".join(filter(lambda x: x != ",", db1_tables))
db1_tables = "".join(filter(lambda x: x != "'", db1_tables)) 

db1_tables = db1_tables.split()

print(db1_tables)


['active_list', 'active_list_allergy', 'active_list_problem', 'active_list_type', 'appframework_component_state', 'appframework_user_app', 'appointmentscheduling_appointment', 'appointmentscheduling_appointment_block', 'appointmentscheduling_appointment_request', 'appointmentscheduling_appointment_status_history', 'appointmentscheduling_appointment_type', 'appointmentscheduling_block_type_map', 'appointmentscheduling_time_slot', 'calculation_registration', 'care_setting', 'clob_datatype_storage', 'cohort', 'cohort_member', 'concept', 'concept_answer', 'concept_class', 'concept_complex', 'concept_datatype', 'concept_description', 'concept_map_type', 'concept_name', 'concept_name_tag', 'concept_name_tag_map', 'concept_numeric', 'concept_proposal', 'concept_proposal_tag_map', 'concept_reference_map', 'concept_reference_source', 'concept_reference_term', 'concept_reference_term_map', 'concept_set', 'concept_set_derived', 'concept_state_conversion', 'concept_stop_word', 'concept_word', 'dhi

#####  Tables in Database 2

In [44]:
cursor = condb.cursor() 
cursor.execute("use %s" % db_two)
cursor.execute("show tables")

db2_tables = cursor.fetchall()
 
db2_tables = ' '.join(str(v) for v in db2_tables)
db2_tables = "".join(filter(lambda x: x != "(", db2_tables)) 
db2_tables = "".join(filter(lambda x: x != ")", db2_tables)) 
db2_tables = "".join(filter(lambda x: x != ",", db2_tables))
db2_tables = "".join(filter(lambda x: x != "'", db2_tables)) 

db2_tables = db2_tables.split()

print(db2_tables)


['active_list', 'active_list_allergy', 'active_list_problem', 'active_list_type', 'appframework_component_state', 'appframework_user_app', 'appointmentscheduling_appointment', 'appointmentscheduling_appointment_block', 'appointmentscheduling_appointment_request', 'appointmentscheduling_appointment_status_history', 'appointmentscheduling_appointment_type', 'appointmentscheduling_block_type_map', 'appointmentscheduling_time_slot', 'calculation_registration', 'care_setting', 'clob_datatype_storage', 'cohort', 'cohort_member', 'concept', 'concept_answer', 'concept_class', 'concept_complex', 'concept_datatype', 'concept_description', 'concept_map_type', 'concept_name', 'concept_name_tag', 'concept_name_tag_map', 'concept_numeric', 'concept_proposal', 'concept_proposal_tag_map', 'concept_reference_map', 'concept_reference_source', 'concept_reference_term', 'concept_reference_term_map', 'concept_set', 'concept_set_derived', 'concept_state_conversion', 'concept_stop_word', 'concept_word', 'dhi

#### Tables Matched In Both Databases

In [45]:
 def matched_tables(a,b):
    included_tables = []
    for table in a:
        if table in b:
            included_tables.append(table.strip())
    return included_tables


In [46]:
inlcuded_tables = matched_tables(db1_tables, db2_tables)
print(inlcuded_tables)

['active_list', 'active_list_allergy', 'active_list_problem', 'active_list_type', 'appframework_component_state', 'appframework_user_app', 'appointmentscheduling_appointment', 'appointmentscheduling_appointment_block', 'appointmentscheduling_appointment_request', 'appointmentscheduling_appointment_status_history', 'appointmentscheduling_appointment_type', 'appointmentscheduling_block_type_map', 'appointmentscheduling_time_slot', 'calculation_registration', 'care_setting', 'clob_datatype_storage', 'cohort', 'cohort_member', 'concept', 'concept_answer', 'concept_class', 'concept_complex', 'concept_datatype', 'concept_description', 'concept_map_type', 'concept_name', 'concept_name_tag', 'concept_name_tag_map', 'concept_numeric', 'concept_proposal', 'concept_proposal_tag_map', 'concept_reference_map', 'concept_reference_source', 'concept_reference_term', 'concept_reference_term_map', 'concept_set', 'concept_set_derived', 'concept_state_conversion', 'concept_stop_word', 'concept_word', 'dhi

##### Create Database For Merging

In [47]:
# merge_sync ="synced_db"
merge_sync = db_one + "_and_" + db_two
cursor.execute("CREATE DATABASE IF NOT EXISTS %s;" % merge_sync)
print(merge_sync)

gucha_daily_and_gucha_migrated


##### Define Connection For Commiting Tables to Database

In [48]:
mysql_connection = pymysql.connect(host='localhost',
                    user='root',
                    password='P@55w0rd.',
                    db='merged',
                    port = 3306,
                    charset='utf8',
                    cursorclass=pymysql.cursors.DictCursor)
engine = create_engine("mysql+mysqldb://root:P@55w0rd.@localhost/%s" % merge_sync)

##### Query The Databeses, Merge Recursively and Commit To Database

In [50]:
cursor.execute("SET @@global.sql_mode= ''")

table_count = 1

for table in inlcuded_tables:
    df = pd.read_sql(f"SELECT * FROM {db_one}.{table} UNION SELECT * FROM {db_two}.{table}", condb)
    df.to_sql(table, con=engine, if_exists='replace', index=False, chunksize=1000000)
    print(str(table_count) + ". " + table + " merged and committed to database successfully")
    table_count += 1
print("===> All matching databases merged and committed to database successfully! <===")
 

1. active_list merged and committed to database successfully
2. active_list_allergy merged and committed to database successfully
3. active_list_problem merged and committed to database successfully
4. active_list_type merged and committed to database successfully
5. appframework_component_state merged and committed to database successfully
6. appframework_user_app merged and committed to database successfully
7. appointmentscheduling_appointment merged and committed to database successfully
8. appointmentscheduling_appointment_block merged and committed to database successfully
9. appointmentscheduling_appointment_request merged and committed to database successfully
10. appointmentscheduling_appointment_status_history merged and committed to database successfully
11. appointmentscheduling_appointment_type merged and committed to database successfully
12. appointmentscheduling_block_type_map merged and committed to database successfully
13. appointmentscheduling_time_slot merged and c

#### Table in Database First Database But Not In Second Database

In [51]:
 def tables_only_in_a(a,b):
    tables_only_in_db_one = []
    for table in a:
        if table not in b:
            tables_only_in_db_one.append(table.strip())
    return tables_only_in_db_one

In [52]:
tables_only_in_db_one = tables_only_in_a(db1_tables, db2_tables)
print(tables_only_in_db_one)

[]


#### Query and Commit Tables Only Present In DB One

In [53]:
cursor.execute("SET @@global.sql_mode= ''")

table_count = 1

for table in tables_only_in_db_one:
    df = pd.read_sql(f"SELECT * FROM {db_one}.{table}", condb)
    df.to_sql(table, con=engine, if_exists='replace', index=False, chunksize=1000000)
    print(str(table_count) + ". " + table + " merged and committed to database successfully")
    table_count += 1
print("===> All tables in DB 1 and not in DB 2 committed to database successfully! <===")
 

===> All tables in DB 1 and not in DB 2 committed to database successfully! <===


#### Table in Database First Database But Not In Second Database

In [54]:
def tables_only_in_b(a,b):
    tables_only_in_db_two = []
    for table in b:
        if table not in a:
            tables_only_in_db_two.append(table.strip())
    return tables_only_in_db_two

In [55]:
tables_only_in_db_two = tables_only_in_b(db1_tables, db2_tables)
print(tables_only_in_db_two)

['il_message_archive', 'il_message_error_queue', 'il_registration', 'medic_archive_data', 'medic_data_source', 'medic_error_data', 'medic_error_message', 'medic_form', 'medic_notification_data', 'medic_queue_data', 'medic_registration_data']


#### Query and Commit Tables Only Present In DB Two

In [56]:
cursor.execute("SET @@global.sql_mode= ''")

table_count = 1

for table in tables_only_in_db_two:
    df = pd.read_sql(f"SELECT * FROM {db_two}.{table}", condb)
    df.to_sql(table, con=engine, if_exists='replace', index=False, chunksize=1000000)
    print(str(table_count) + ". " + table + " merged and committed to database successfully")
    table_count += 1
print("===> All tables in DB 2 and not in DB 1 committed to database successfully! <===")
 

1. il_message_archive merged and committed to database successfully
2. il_message_error_queue merged and committed to database successfully
3. il_registration merged and committed to database successfully
4. medic_archive_data merged and committed to database successfully
5. medic_data_source merged and committed to database successfully
6. medic_error_data merged and committed to database successfully
7. medic_error_message merged and committed to database successfully
8. medic_form merged and committed to database successfully
9. medic_notification_data merged and committed to database successfully
10. medic_queue_data merged and committed to database successfully
11. medic_registration_data merged and committed to database successfully
===> All tables in DB 2 and not in DB 1 committed to database successfully! <===
